In [ ]:
import pandas as pd
import numpy as np

import copy
import pickle
from datetime import datetime

import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import median_absolute_error as MAPE
from sklearn.metrics import r2_score  as R2
from xgboost import plot_importance
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

from sklearn.metrics import r2_score

# Parameter tunning
from sklearn.model_selection import KFold


import seaborn as sns
import matplotlib.pyplot as plt 

import optuna

import shap

from google.colab import drive

%matplotlib inline

In [ ]:
def date_to_year(date_string):
    fmt = '%Y-%m-%d'
    return datetime.strptime(date_string,fmt).year

In [ ]:
# Execute this cell paste the Key, and press the key ENTER...
drive.mount('/content/drive/')

In [ ]:
Data = pd.read_csv('/content/drive/My Drive/datasetV6.csv') 

### Crime

In [ ]:
Data['crime_total']=Data['PublicOrder']+Data['Burglary']+Data['Robbery']+Data['PossessionOfWeapons']+Data['BicycleTheft']+Data['AntiSocialBehaviour']+Data['Violence']+Data['Theft']+Data['Shoplifting']+Data['OtherCrime']+Data['Drugs']+Data['OtherTheft']+Data['VehicleCrime']+Data['CriminalDamage']

In [ ]:
Data['PublicOrder']=Data['PublicOrder']/Data['crime_total']

In [ ]:
Data['Burglary'] = Data['Burglary']/Data['crime_total']
Data['Robbery'] = Data['Robbery']/Data['crime_total']
Data['PossessionOfWeapons'] = Data['PossessionOfWeapons']/Data['crime_total']
Data['BicycleTheft'] = Data['BicycleTheft']/Data['crime_total']
Data['AntiSocialBehaviour'] = Data['AntiSocialBehaviour']/Data['crime_total']
Data['Violence'] = Data['Violence']/Data['crime_total']
Data['Theft'] = Data['Theft']/Data['crime_total']
Data['Shoplifting'] = Data['Shoplifting']/Data['crime_total']
Data['OtherCrime'] = Data['OtherCrime']/Data['crime_total']
Data['Drugs'] = Data['Drugs']/Data['crime_total']
Data['OtherTheft'] = Data['OtherTheft']/Data['crime_total']
Data['VehicleCrime'] = Data['VehicleCrime']/Data['crime_total']
Data['CriminalDamage'] = Data['CriminalDamage']/Data['crime_total']

Data['crime_Type_A'] = Data['Robbery']+Data['Drugs']+Data['PossessionOfWeapons']+Data['Burglary']+Data['OtherCrime']
Data['crime_Type_B'] = Data['OtherTheft'] + Data['Shoplifting'] + Data['Theft'] + Data['BicycleTheft']
Data['crime_Type_C'] = Data['CriminalDamage']+ Data['Violence'] + Data['AntiSocialBehaviour']
Data['crime_Type_D'] = Data['VehicleCrime']

In [ ]:
dropList=['Burglary',
 'Robbery',
 'PossessionOfWeapons',
 'BicycleTheft',
 'AntiSocialBehaviour',
 'Violence',
 'Theft',
 'Shoplifting',
 'OtherCrime',
 'Drugs',
 'OtherTheft',
 'VehicleCrime',
 'CriminalDamage',]
Data.drop(dropList, axis = 1, inplace = True)

### Age 


In [ ]:
Data['TotalModPop'] = Data['age20_24']+Data['age25_29']+Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age45_49']+Data['age50_54']+Data['age60_64']
Data['age20_29'] = Data['age20_24']+Data['age25_29']
Data['age30_64'] = Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age50_54']+Data['age60_64']
Data['age65_'] = Data['age70_74']+Data['age75_79']+Data['age80_84']+Data['age85_89']

In [ ]:
dropList=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',
 'age45_49',
 'age50_54',
 'age60_64',
 'age70_74',
 'age75_79',
 'age80_84',
 'age85_89']
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
Data['commute_method_public'] = Data['commute_method_train'] + Data['commute_method_bus'] + Data['commute_method_underground_light_rail']
Data['commute_method_private'] = Data['commute_method_motorcycle'] + Data['commute_method_taxi'] + Data['commute_method_car_driver'] + Data['commute_method_car_passenger']
Data['commute_method_open'] = Data['commute_method_foot']+Data['commute_method_bicycle']

In [ ]:
dropList=['commute_method_foot',
 'commute_method_bicycle',
 'commute_method_other',
 'commute_method_motorcycle',
 'commute_method_taxi',
 'commute_method_train',
 'commute_method_bus',
 'commute_method_underground_light_rail',
 'commute_method_car_driver',
 'commute_method_at_home',
 'commute_method_car_passenger',
]
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
dropList=[ 'dyn_sold_points_analysed',
 'dyn_sold_70pc_rangeMIN',
 'dyn_sold_70pc_rangeMAX',
 'dyn_sold_80pc_rangeMIN',
 'dyn_sold_80pc_rangeMAX',
 'dyn_sold_90pc_rangeMIN',
 'dyn_sold_90pc_rangeMAX',
 'dyn_sold_sqf_points_analysed',
 'dyn_sold_sqf_70pc_rangeMIN',
 'dyn_sold_sqf_70pc_rangeMAX',
 'dyn_sold_sqf_80pc_rangeMIN',
 'dyn_sold_sqf_80pc_rangeMAX',
 'dyn_sold_sqf_90pc_rangeMIN',
 'dyn_sold_sqf_90pc_rangeMAX',
 'dyn_rents_points_analysed',
 'dyn_rents_70pc_rangeMIN',
 'dyn_rents_70pc_rangeMAX',
 'dyn_rents_80pc_rangeMIN',
 'dyn_rents_80pc_rangeMAX',
 'dyn_rents_90pc_rangeMIN',
 'dyn_rents_90pc_rangeMAX',]
Data.drop(dropList, axis = 1, inplace = True)

## POI

In [ ]:
Data['HperP']=Data['Households']/Data['Population']

In [ ]:
total = Data['construction_before_1900']+Data['construction_1900_1929']+Data['construction_1930_1949']+Data['construction_1950_1966']+Data['construction_1967_1975']+Data['construction_1976_1982']+Data['construction_1983_1990']+Data['construction_1991_1995']+Data['construction_1996_2002']+Data['construction_2003_2006']+Data['construction_2007_2011']+Data['construction_2012_onwards']

Data['construction_before_1950']=(Data['construction_before_1900'] + Data['construction_1900_1929'] + Data['construction_1930_1949']) /total
Data['construction_1950_1975']=(Data['construction_1950_1966']+Data['construction_1967_1975']+ Data['construction_1976_1982'])/total
Data['construction_1983_1990']=(Data['construction_1983_1990']+Data['construction_1991_1995'])/total
Data['construction_1996_2006']=(Data['construction_1996_2002']+Data['construction_2003_2006'])/total
Data['construction_2007_onwards']=(Data['construction_2007_2011']+Data['construction_2012_onwards'])/total

Data['count'] = Data['count']/Data['Households']

In [ ]:
dropList=['construction_before_1900',
 'construction_1900_1929',
 'construction_1930_1949',
 'construction_1950_1966',
 'construction_1967_1975',
 'construction_1976_1982',
 'construction_1983_1990',
 'construction_1991_1995',
 'construction_1996_2002',
 'construction_2003_2006',
 'construction_2007_2011',
 'construction_2012_onwards']
Data.drop(dropList, axis = 1, inplace = True)

Removing irrelevant attributes

In [ ]:
dropList=['Unnamed: 0','postcode','count',
         'postcode_Latitude','postcode_Longitude','MSOA_Code','LSOA_Code','area'
          ]
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
dropList=['dyn_sold_average',
 'dyn_sold_100pc_rangeMIN',
 'dyn_sold_100pc_rangeMAX',
 'dyn_sold_sqf_100pc_rangeMIN',
 'dyn_sold_sqf_100pc_rangeMAX',
 'dyn_rents_average',
 'dyn_rents_100pc_rangeMIN',
 'dyn_rents_100pc_rangeMAX',
         ]
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
len(Data)

Adding the price paid by sqf

In [ ]:
sns.set_style('whitegrid')
sns.jointplot(x ='population', y ='Population', data = Data, kind ='kde')
plt.legend(title='Area population vs Postcode polulation')

*italicized text*### sold_sqf_average statistics

In [ ]:
Data['dyn_sold_sqf_average'].describe()

In [ ]:
sns.set_style('whitegrid')
sns.displot(Data['dyn_sold_sqf_average'], kde = False, color ='red', bins = 30)
plt.legend(title='sold_sqf_average paid distribution')

In [ ]:
sns.set_style('whitegrid')
sns.displot(Data['Average_Income'], kde = False, color ='red', bins = 30)
plt.legend(title='Postcode avarage income distribution')

### Categorical attribute binarization

In [ ]:
lbl = preprocessing.LabelEncoder()

catData = ['dyn_ptal', 'dyn_flood_risk','crime_rating']
for att in catData:
    unique_classes = Data[att].unique()
    one_hot = pd.get_dummies(unique_classes, prefix=att,dummy_na=True, columns = [att])
    one_hot[att] = unique_classes
    Data = Data.merge(one_hot, on = [att], how='left')
    Data = Data.drop(columns = [att])

In [ ]:
list(Data.columns)

### Define sold_sqf_average our goal 

In [ ]:
Data['price'] = Data['dyn_sold_sqf_average']
Data.drop(['dyn_sold_sqf_average'], axis = 1, inplace = True)

In [ ]:
# Split data
X, y = Data.iloc[:, :-1], Data.iloc[:, -1]

### Attributes used for the model

In [ ]:
def objective(trial, X, y, kf):
    n_estimators = trial.suggest_int('n_estimators', 206, 512)
    learning_rate =trial.suggest_uniform('learning_rate', 0, 0.1) 
    #max_depth = trial.suggest_int('max_depth', 3, 10)
    min_split_loss = trial.suggest_uniform('min_split_loss', 0, 2.5)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    reg_lambda = trial.suggest_uniform('reg_lambda', 0, 10)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0, 10)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1)
    eta = trial.suggest_uniform('eta', 0, 1)
    
    model = xg.XGBRegressor(n_estimators=n_estimators,
                          #max_depth=max_depth,
                          learning_rate=learning_rate,
                          min_split_loss=min_split_loss,
                          min_child_weight=min_child_weight,
                          reg_lambda=reg_lambda,
                          reg_alpha=reg_alpha,
                          colsample_bytree=colsample_bytree,
                          subsample=subsample,
                          eta=eta,
                          objective='reg:squarederror', 
                          booster='gbtree',
                          seed=42,
                          n_jobs=3)
    
    
    test_score = []
    
    step = 0
    
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        test_score.append(MAE(y_test, y_pred)/739.302041*100)
        #test_score.append(R2(y_test, y_pred))
        
        trial.report(np.mean(test_score), step)
        
        if trial.should_prune():
            raise optuna.TrialPruned()
            
        step+=1

    return np.mean(test_score)



kf = KFold(n_splits=4, shuffle=True, random_state=42)

study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.SuccessiveHalvingPruner())

study.optimize(lambda trial: objective(trial, X, y, kf), timeout=30*60, n_jobs=1)

study.best_params 

In [ ]:


# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.2)

In [ ]:
model_xg = XGBRegressor(objective='reg:squarederror', 
                     n_estimators= 406,
                     learning_rate= 0.06626130240901489,
                     min_split_loss= 1.4502808579693633,
                     min_child_weight= 3,
                     reg_lambda= 6.047816441210893,
                     reg_alpha= 1.4218025841002861,
                     colsample_bytree= 0.9180203712392442,
                     subsample=   0.8654788672331599,
                     eta= 0.9998708954001517)

xgb_r = model_xg.fit(train_X, train_y)

pred_train = model_xg.predict(train_X)
pred_test = model_xg.predict(test_X)

# MAE Computation
mae =MAE(train_y, pred_train)
print("Train MAE : % f --- %f" %(mae,mae/739.302041*100),"%")
# MAE Computation
mae =MAE(test_y, pred_test)
print("Test MAE : % f --- %f" %(mae,mae/739.302041*100),"%")

# MAE Computation
mape =MAPE(train_y, pred_train)
print("Train MAPE : % f --- %f" %(mape,mape/739.302041*100),"%")
# MAE Computation
mape =MAPE(test_y, pred_test)
print("Test MAPE : % f --- %f" %(mape,mape/739.302041*100),"%")

# MAE Computation
r2 =R2(train_y, pred_train)
print("Train R2 : % f" %(r2))
# MAE Computation
r2 =R2(test_y, pred_test)
print("Test R2 : % f" %(r2))

## Ranking by weight and select 30

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17,40))

xg.plot_importance(xgb_r, ax,importance_type='weight')

In [ ]:
results=pd.DataFrame()
results['columns']=X.columns

results['importances'] = xgb_r.feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)


selected= list(results['columns'][:30])


X = X[selected].copy()

In [ ]:
X.isna().sum().sort_values(ascending=False)

In [ ]:
X.dropna(axis=0, how='any', inplace=True)
y = y.loc[X.index]

In [ ]:
train_dataset = X.copy()
train_dataset.insert(0, "Price", y)
_ = sns.pairplot(
    train_dataset[ selected[:10]],
    kind='reg', diag_kind='kde', plot_kws={'scatter_kws': {'alpha': 0.1}})

In [ ]:
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.2)

### Linear model inspection

In [ ]:
from sklearn.linear_model import RidgeCV

model = RidgeCV()

model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

Linear model obtains a score of .80, so it explains a significant part of the target. Its coefficient should be somehow relevant. Let’s look at the coefficients

In [ ]:
coefs = pd.DataFrame(
   model.coef_,
   columns=['Coefficients'], index=train_X.columns
)

coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

## Sign of coefficients

The coefficients of a linear model are a conditional association: they quantify the variation of a the output (the price) when the given feature is varied, keeping all other features constant. We should not interpret them as a marginal association, characterising the link between the two quantities ignoring all the rest.


In [ ]:
train_X.std(axis=0).plot(kind='barh', figsize=(9, 7))
plt.title('Features std. dev.')
plt.subplots_adjust(left=.3)
plt.xlim((0, 100))

We need to scale each column (removing the mean and scaling the variance to 1).

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), RidgeCV())

model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

In [ ]:
coefs = pd.DataFrame(
   model[1].coef_,
   columns=['Coefficients'], index=train_X.columns
)

coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

### Checking the variability of the coefficients

We can check the coefficient variability through cross-validation.

If coefficients vary significantly when changing the input dataset their robustness is not guaranteed, and they should probably be interpreted with caution.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold

cv_model = cross_validate(
   model, X, y, cv=RepeatedKFold(n_splits=5, n_repeats=5),
   return_estimator=True, n_jobs=2
)
coefs = pd.DataFrame(
   [model[1].coef_
    for model in cv_model['estimator']],
   columns=X.columns
)
plt.figure(figsize=(9, 7))
sns.boxplot(data=coefs, orient='h', color='cyan', saturation=0.5)
plt.axvline(x=0, color='.5')
plt.xlabel('Coefficient importance')
plt.title('Coefficient importance and its variability')
plt.subplots_adjust(left=.3)

Coefficient are not stable.

In [ ]:
# Linear models with sparse coefficients (Lasso)

In it important to keep in mind that the associations extracted depend on the model. To illustrate this point we consider a Lasso model, that performs feature selection with a L1 penalty. Let us fit a Lasso model with a strong regularization parameters alpha

In [ ]:
from sklearn.linear_model import Lasso

model = make_pipeline(StandardScaler(), Lasso(alpha=.5,max_iter=100000))

model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

In [ ]:
coefs = pd.DataFrame(
   model[1].coef_,
   columns=['Coefficients'], index=train_X.columns
)

coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Lasso model, strong regularization')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

Here the model score is a bit lower, because of the strong regularization. However, it has zeroed out 4 coefficients, selecting a small number of variables to make its prediction.
.

As above, we can look at the variability of the coefficients

In [ ]:
cv_model = cross_validate(
   model, X, y, cv=RepeatedKFold(n_splits=5, n_repeats=5),
   return_estimator=True, n_jobs=2
)
coefs = pd.DataFrame(
   [model[1].coef_
    for model in cv_model['estimator']],
   columns=X.columns
)
plt.figure(figsize=(9, 7))
sns.boxplot(data=coefs, orient='h', color='cyan', saturation=0.5)
plt.axvline(x=0, color='.5')
plt.xlabel('Coefficient importance')
plt.title('Coefficient importance and its variability')
plt.subplots_adjust(left=.3)

We can see that both the coefficients associated to subway_station_rating_mean_1000, subway_station_count_1500, crimes_per_thousand  and health have a strong variability and that they can both be non zero. Given that they are strongly correlated, the model can pick one or the other to predict well. This choice is a bit arbitrary, and must not be over-interpreted.

In [ ]:
train_dataset = X.copy()
train_dataset.insert(0, "Price", y)
_ = sns.pairplot(
    train_dataset[['subway_station_rating_mean_1000','subway_station_count_1500','crimes_per_thousand']],
    kind='reg', diag_kind='kde', plot_kws={'scatter_kws': {'alpha': 0.1}})

## RandomForest

On some algorithms, there pre-exist some feature importance method, inherently built within the model. It is the case in RandomForest models. Let’s investigate the built-in feature_importances_ attribute.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

Contrary to the testing set, the score on the training set is almost perfect, which means that our model is overfitting here.

In [ ]:
importances = model.feature_importances_

The importance of a feature is basically: how much this feature is used in each tree of the forest. Formally, it is computed as the (normalized) total reduction of the criterion brought by that feature.

In [ ]:
indices = np.argsort(importances)

fig, ax = plt.subplots(figsize=(9, 7))
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(train_X.columns)[indices])

## Random Forest Regressor: Feature importance by permutation

We introduce here a new technique to evaluate the feature importance of any given fitted model. It basically shuffles a feature and sees how the model changes its prediction. Thus, the change in prediction will be correspond to the feature importance.

In [ ]:
# Any model could be used here

model = RandomForestRegressor()

In [ ]:
model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

As the model gives a good prediction, it has captured well the link between X and y. Hence, it is reasonable to interpret what it has captured from the data.

## Feature importance

Lets compute the feature importance for a given feature, say the restaurant_hygiene.

For that, we will shuffle this specific feature, keeping the other feature as is, and run our same model (already fitted) to predict the outcome. The decrease of the score shall indicate how the model had used this feature to predict the target. The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled

On the contrary, if the feature is not used by the model, the score shall remain the same, thus the feature importance will be close to 0.

In [ ]:
def get_score_after_permutation(model, X, y, curr_feat):
    """ return the score of model when curr_feat is permuted """

    X_permuted = X.copy()
    col_idx = list(X.columns).index(curr_feat)
    # permute one column
    X_permuted.iloc[:, col_idx] = np.random.permutation(
        X_permuted[curr_feat].values)

    permuted_score = model.score(X_permuted, y)
    return permuted_score


def get_feature_importance(model, X, y, curr_feat):
    """ compare the score when curr_feat is permuted """

    baseline_score_train = model.score(X, y)
    permuted_score_train = get_score_after_permutation(model, X, y, curr_feat)

    # feature importance is the difference between the two scores
    feature_importance = baseline_score_train - permuted_score_train
    return feature_importance


In [ ]:
curr_feat='subway_station_min'

feature_importance = get_feature_importance(model, train_X, train_y, curr_feat)
print(f'feature importance of "{curr_feat}" on train set is'
      f'{feature_importance:.3}')

Since there are some randomness, it is advice to run multiple times and inspect the mean and the standard deviation of the feature importance

In [ ]:
n_repeats = 10

list_feature_importance = []
for n_round in range(n_repeats):
    list_feature_importance.append(
        get_feature_importance(model, train_X, train_y, curr_feat))

print(
    f'feature importance of "{curr_feat}" on train set is '
    f'{np.mean(list_feature_importance):.3} '
    f'+/- {np.std(list_feature_importance):.3}')

0.91 over .06 is very relevant. So we can imagine our model relies heavily on this feature to predict the class. We can now compute the feature permutation importance for all the features

In [ ]:
def permutation_importance(model, X, y, n_repeats=10):
    """Calculate importance score for each feature."""

    importances = []
    for curr_feat in X.columns:
        list_feature_importance = []
        for n_round in range(n_repeats):
            list_feature_importance.append(
                get_feature_importance(model, X, y, curr_feat))

        importances.append(list_feature_importance)

    return {'importances_mean': np.mean(importances, axis=1),
            'importances_std': np.std(importances, axis=1),
            'importances': importances}

# This function could directly be access from sklearn
# from sklearn.inspection import permutation_importance

In [ ]:
def plot_importantes_features(perm_importance_result, feat_name):
    """ bar plot the feature importance """

    fig, ax = plt.subplots(figsize=(17,20))
    
    indices = perm_importance_result['importances_mean'].argsort()
    plt.barh(range(len(indices)),
             perm_importance_result['importances_mean'][indices],
             xerr=perm_importance_result['importances_std'][indices])

    ax.set_yticks(range(len(indices)))
    _ = ax.set_yticklabels(feat_name[indices])


Let’s compute the feature importance by permutation on the training data.

In [ ]:
perm_importance_result_train = permutation_importance(model, train_X, train_y, n_repeats=10)

plot_importantes_features(perm_importance_result_train, train_X.columns)

## Single regression tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(train_X, train_y)

print(f'model score on training data: {model.score(train_X, train_y)}')
print(f'model score on testing data: {model.score(test_X, test_y)}')

In [ ]:
curr_feat='subway_station_min'

feature_importance = get_feature_importance(model, train_X, train_y, curr_feat)
print(f'feature importance of "{curr_feat}" on train set is'
      f'{feature_importance:.3}')

In [ ]:
n_repeats = 10

list_feature_importance = []
for n_round in range(n_repeats):
    list_feature_importance.append(
        get_feature_importance(model, train_X, train_y, curr_feat))

print(
    f'feature importance of "{curr_feat}" on train set is '
    f'{np.mean(list_feature_importance):.3} '
    f'+/- {np.std(list_feature_importance):.3}')

In [ ]:
perm_importance_result_train = permutation_importance(model, train_X, train_y, n_repeats=10)

plot_importantes_features(perm_importance_result_train, train_X.columns)

In [ ]:
## XGBoost

## XGboost

In [ ]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)

study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.SuccessiveHalvingPruner())

study.optimize(lambda trial: objective(trial, X, y, kf), timeout=30*60, n_jobs=1)

study.best_params 

In [ ]:
model = XGBRegressor(objective='reg:squarederror', 
                     n_estimators= 495,
                     learning_rate= 0.09990364840682915,
                     min_split_loss= 2.243975737143064,
                     min_child_weight= 2,
                     reg_lambda= 0.20535373157835968,
                     reg_alpha= 8.4151064743924,
                     colsample_bytree= 0.9817246411218696,
                     subsample=   0.5108085648853521,
                     eta= 0.3922553995611887)

xgb_r = model_xg.fit(train_X, train_y)

In [ ]:
print(f'model score on training data: {model_xg.score(train_X, train_y)}')
print(f'model score on testing data: {model_xg.score(test_X, test_y)}')

In [ ]:
curr_feat='subway_station_min'

feature_importance = get_feature_importance(model_xg, train_X, train_y, curr_feat)
print(f'feature importance of "{curr_feat}" on train set is'
      f'{feature_importance:.3}')

In [ ]:
n_repeats = 10

list_feature_importance = []
for n_round in range(n_repeats):
    list_feature_importance.append(
        get_feature_importance(model_xg, train_X, train_y, curr_feat))

print(
    f'feature importance of "{curr_feat}" on train set is '
    f'{np.mean(list_feature_importance):.3} '
    f'+/- {np.std(list_feature_importance):.3}')

In [ ]:
perm_importance_result_train = permutation_importance(model_xg, train_X, train_y, n_repeats=10)

plot_importantes_features(perm_importance_result_train, train_X.columns)